In [1]:
using Pkg; Pkg.activate(".")
using ReverseDiff, ForwardDiff, Optimisers, Zygote, 
      LinearAlgebra, PrettyPrinting, NamedTupleTools

  Activating project at `~/gits/PACMAN_ADworkshop/Day1`


This notebook implements a very simple multi-layer perceptron (MLP) 
to illustrate how backpropagation is implemented. 

The MLP is a function $f : \mathbb{R}^{N_{\rm in}} \to \mathbb{R}$ defined as 
$$
  f(x) = W_3 \cdot \big( W_2 \sigma( W_1 x+ b_1 ) + b_2 \big) + b_3
$$
where $W_1 \in \mathbb{R}^{N_1 \times N_{\rm in}}, W_2 \in \mathbb{R}^{N_2 \times N_1}, W_3 \in \mathbb{R}^{N_2}$ and the $b_i$ analogous. It is more conveniently expressed in stages, 
$$
\begin{aligned} 
    x_1 &= W_1 x+ b_1 \\ 
    x_2 &= W_2 x_1 + b_2 \\ 
    f(x) = y &= W_3 \cdot x_2 + b_3.
\end{aligned}
$$

In [2]:

# define the sigmoid and its derivative. We could of course 
# evaluate the derivative by hand, but we'll use ForwardDiff.
# It is convenient and there is no performance penalty.
σ(x) = 1 / (1 + exp(x))
dσ(x) = ForwardDiff.derivative(σ, x)

"""
Define a simple MLP with 2 hidden layers:

   mlp(x) =  W₃ ⋅ σ(W₂ ⋅ σ(W₁ ⋅ x + b₁) + b₂) + b₁
"""
function mlp(x, p) 
   x1 = σ.(p.W1 * x + p.b1)
   x2 = σ.(p.W2 * x1 + p.b2)
   return dot(p.W3, x2) + p.b3
end


"""
A simple utility function to generate inputs for `mlp`.
"""
function mlp_init(Nin, N1, N2) 
   x = randn(Nin)
   p = (W1 = randn(N1, Nin), b1 = randn(N1), 
        W2 = randn(N2, N1),  b2 = randn(N2), 
        W3 = randn(N2),      b3 = randn() )
   return x, p        
end


mlp_init

In [3]:
# evaluting the mlp gives a scalar output. 
Nin = 3; N1 = 4; N2 = 2
x, p = mlp_init(Nin, N1, N2)
mlp(x, p)

1.0695008358878582

In [4]:
# AD will "automatically" give us the gradient 
# w.r.t. the input and/or w.r.t. the parameters.
∇x_ad = Zygote.gradient(_x -> mlp(_x, p), x)[1]
∇p_ad = Zygote.gradient(_p -> mlp(x, _p), p)[1]

print("∇x = "); pprintln(∇x_ad)
println("-"^80)
print("∇p = "); pprint(∇p_ad)

∇x = [0.047955891559071415, -0.021675281122803536, 0.01034124616918743]
--------------------------------------------------------------------------------
∇p = (W1 =
     [0.04528112443269612 0.007323798408162094 -0.010799920402466226; -0.0631973165530103 -0.010221574931492798 0.015073079499968008; 0.008435405462189322 0.0013643479456436618 -0.0020119135444523484; 0.0034269209949327807 0.0005542724223840916 -0.0008173488276737725],
 b1 = [0.02035796579242188,
       -0.0284129165226747,
       0.0037924786099273527,
       0.0015407113065813935],
 W2 =
     [-0.007362584416253665 -0.05957602220615831 -0.0062375722056526375 -0.0028628691326680776; -0.015267495870337298 -0.12354040668052114 -0.012934603191848998 -0.005936616843921574],
 b2 = [-0.10026364721322245, -0.20791270201174006],
 W3 = [0.27098364549387494, 0.2481311529932934],
 b3 = 1.0)

What AD (Zygote in this case) does is it generates a new function that 
first executes the original function (forward pass) but then adds additional
steps - the backward pass - that accumulate the gradient information
The following function is indicative of how this generated code might look
like if we were to inspect it. 

In high level terms it looks like this: 
$$
\begin{aligned} 
        \text{FORWARD PASS} &   \\ 
    x_1 &= f_1(x, W_1, b_1) = W_1 x+ b_1 \\ 
    x_2 &= f_2(x_1, W_2, B_2) = W_2 x_1 + b_2 \\ 
    f(x) = y &= f_3(x_2, W_3, b_3) = W_3 \cdot x_2 + b_3. \\[3mm]
    \text{BACKWARD PASS} &  \\ 
    \partial_{x_2} f &= \partial_{x_2} f_3 = W_3 \\ 
    \partial_{W_3} f &= \partial_{W_3} f_3 = x_2 \\ 
    \partial_{b_3} f &= \partial_{b_3} f_3 = 1  \\[2mm] 
    \partial_{x_1} f &= \partial_{x_2} f \cdot \partial_{x_1} x_2 
                        = \partial_{x_2} f \cdot \partial_{x_1} f_2 = W_2^T \big(\partial_{x_2} f \odot \sigma'(W_2 x_1 + b_2) \big)  \\ 
    \partial_{W_2} f &= \dots  \text{(similar calculation)} \\ 
    \partial_{b_2} f &= \dots \\ 
    \partial_{x} f &= \partial_{x_1} f \cdot \partial_{x} x_1 
                        = \partial_{x_1} f \cdot \partial_{x} f_1 = W_1^T \big(\partial_{x_1} f \odot \sigma'(W_1 x + b_1) \big) \\ 
    \partial_{W_1} f &= \dots \\ 
    \partial_{b_1} f &= \dots
\end{aligned}
$$
where $\odot$ denotes the element-wise multiplication (Hadamard product). After working out all the expressions we can implement this as follows:


In [5]:

function mlp_withgrad(x, p)
   # unpack the parameters 
   W1 = p.W1; b1 = p.b1 
   W2 = p.W2; b2 = p.b2 
   W3 = p.W3; b3 = p.b3
   
   # Forward Pass
   # fwd stage 1
   z1 = W1 * x + b1
   x1 = σ.(z1)   
   # fwd stage 2 
   z2 = W2 * x1 + b2
   x2 = σ.(z2)  
   # fwd stage 3 
   y = dot(W3, x2) + b3    

   # Backward Pass
   # bwd stage 3
   ∂y_∂x2 = W3
   ∂y_∂W3 = x2
   ∂y_∂b3 = 1.0
   # bwd stage 2
   t2 = ∂y_∂x2 .* dσ.(z2)  # N₂ - vector
   ∂y_∂x1 = W2' * t2
   ∂y_∂W2 = t2 * x1'
   ∂y_∂b2 = t2 
   # bwd stage 1
   t1 = ∂y_∂x1 .* dσ.(z1)   # N₁ - vector
   ∂y_∂x = W1' * t1
   ∂y_∂W1 = t1 * x'
   ∂y_∂b1 = t1 

   # pack the gradients into a named tuple (~ static Dict)
   ∂y_∂p = ( W1 = ∂y_∂W1, b1 = ∂y_∂b1, 
             W2 = ∂y_∂W2, b2 = ∂y_∂b2, 
             W3 = ∂y_∂W3, b3 = ∂y_∂b3 )
            
   return y, ∂y_∂x, ∂y_∂p
end;

In [6]:
# We can evaluate the gradients with our hand-written implementation 
# and confirm that they are comparable. 
y, ∇x, ∇p = mlp_withgrad(x, p)
@show y ≈ mlp(x, p)
@show ∇x ≈ ∇x_ad 
@show all(∇p[s] ≈ ∇p_ad[s] for s ∈ fieldnames(∇p));

y ≈ mlp(x, p) = true
∇x ≈ ∇x_ad = true
all((∇p[s] ≈ ∇p_ad[s] for s = fieldnames(∇p))) = true


In [7]:
# To wrap this up, let's just compare the performance of these 
# two implementations.
using BenchmarkTools

println("Performance Evaluation only: ")        
@btime mlp($x, $p)

println("Performance Zygote: ")
@btime Zygote.gradient(_p -> mlp($x, _p), $p);

println("Performance Hand-coded: ")
@btime mlp_withgrad($x, $p);

Performance Evaluation only: 
  183.817 ns (6 allocations: 528 bytes)
Performance Zygote: 
  1.762 μs (53 allocations: 7.16 KiB)
Performance Hand-coded: 
  375.810 ns (12 allocations: 1.14 KiB)


We shouldn't read too much into this performance comparison. There are some ways
to make Zygote a bit faster and there are MANY ways to make the hand-coded 
version faster. 

But still there is an important message. Backward differentiation 
is more than just a useful tool for AD. It is also a means for us to organize 
our algorithms in a systematic way to get the best possible performance. All 
AD tools have overheads, sometimes they are negligible, sometimes they dominate. 
Knowing how to hand-code adjoints and chains/networks of function evaluations 
in a systematic way can sometimes lead to significant performance gains. 

Just to make the point, the following shows a simple trick to improve the 
performance significantly further. We simply replace all arrays with static 
arrays for which the size is known at compile time. This moves the arrays 
onto the stack avoiding all intermediate allocations from system memory. 
What happens is that our simple hand-written code becomes quite fast, 
but the AD system overhead prevents the same speedup for the AD.


In [8]:
using StaticArrays

x_st = SVector{Nin}(x)
p_st = (W1 = SMatrix{N1, Nin}(p.W1), b1 = SVector{N1}(p.b1), 
        W2 = SMatrix{N2, N1}(p.W2), b2 = SVector{N2}(p.b2), 
        W3 = SVector{N2}(p.W3), b3 = p.b3 )

println("Performance Evaluation only with StaticArrays: ")        
@btime mlp($x_st, $p_st)

println("Performance Zygote with StaticArrays: ")
@btime Zygote.gradient(_p -> mlp($x_st, _p), $p_st)

println("Performance hand-coded with StaticArrays: ")
@btime mlp_withgrad($x_st, $p_st);

Performance Evaluation only with StaticArrays: 
  31.228 ns (0 allocations: 0 bytes)
Performance Zygote with StaticArrays: 
  750.362 ns (21 allocations: 880 bytes)
Performance hand-coded with StaticArrays: 
  71.709 ns (0 allocations: 0 bytes)
